In [21]:
import pandas as pd
import numpy as np

import nltk
from nltk.stem.porter import *
import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from wordsegment import load, segment, clean
load() #loading segment
import matplotlib.pyplot as plt

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings('ignore')
#nltk.download()
import csv
from csv import reader
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings
  
warnings.filterwarnings(action = 'ignore')
  
import gensim
from gensim.models import Word2Vec

In [22]:
data = pd.read_csv('abc.csv')
data.head()

,tweets,ClassLabel
0,rt use raya so lovely as a woman you shouldnt ...,2
1,rt user leewboydatscoldtygadwn bad for cuff in...,1
2,rt user r kind of brand dawg rt users baby lif...,1
3,rt user g anderson use riva based she look lik...,1
4,rt user he nika roberts the shit you hear abou...,1


In [23]:
# We'll write a function which will clean the text and prepare it.
def cleanText(text):
    cleaned = re.sub("[^a-zA-Z0-9']"," ",text)
    lowered = cleaned.lower()
    return lowered.strip()

cleanText("Let's test our function, by writing this string!")

"let's test our function  by writing this string"

In [24]:
x,y = np.asarray(data["tweets"]),np.asarray(data["ClassLabel"])

x_cleaned = [cleanText(t) for t in x]
x_cleaned[:4]



['rt use raya so lovely as a woman you shouldnt complain about cleaning up your house and as a man you should always take the trash out',
 'rt user leewboydatscoldtygadwn bad for cuff in dat hoe in the st place',
 'rt user r kind of brand dawg rt users baby life you ever fuck a bitch and she start to cry you be confused as shit',
 'rt user g anderson use riva based she look like a tranny']

In [25]:
# Also we should convert our categories to the integer labels
label_map = {cat:index for index,cat in enumerate(np.unique(y))}
y_prep = np.asarray([label_map[l] for l in y])

label_map



{0: 0, 1: 1, 2: 2}

Word Embedding

In [26]:


x_tokenized = [[w for w in sentence.split(" ") if w != ""] for sentence in x_cleaned]
x_tokenized[0]



['rt',
 'use',
 'raya',
 'so',
 'lovely',
 'as',
 'a',
 'woman',
 'you',
 'shouldnt',
 'complain',
 'about',
 'cleaning',
 'up',
 'your',
 'house',
 'and',
 'as',
 'a',
 'man',
 'you',
 'should',
 'always',
 'take',
 'the',
 'trash',
 'out']

In [27]:
# Now we'll create our model 
import time

start = time.time()

model = gensim.models.Word2Vec(x_tokenized,
                 vector_size=100
                 # Size is the length of our vector.
                )

end = round(time.time()-start,2)
print("This process took",end,"seconds.")

This process took 1.89 seconds.


In [28]:
model.wv.most_similar("hated")



[('threw', 0.978693962097168),
 ('doe', 0.9782025218009949),
 ('lean', 0.9779654741287231),
 ('anyway', 0.9777856469154358),
 ('sooo', 0.9773495197296143),
 ('grade', 0.9772722125053406),
 ('general', 0.9770423769950867),
 ('lost', 0.9762409925460815),
 ('except', 0.9761629104614258),
 ('hmmm', 0.9760681390762329)]

Sequence

In [29]:
class Sequencer():
    
    def __init__(self,
                 all_words,
                 max_words,
                 seq_len,
                 embedding_matrix
                ):
        
        self.seq_len = seq_len
        self.embed_matrix = embedding_matrix
        """
        temp_vocab = Vocab which has all the unique words
        self.vocab = Our last vocab which has only most used N words.
    
        """
        temp_vocab = list(set(all_words))
        self.vocab = []
        self.word_cnts = {}
        """
        Now we'll create a hash map (dict) which includes words and their occurencies
        """
        for word in temp_vocab:
            # 0 does not have a meaning, you can add the word to the list
            # or something different.
            count = len([0 for w in all_words if w == word])
            self.word_cnts[word] = count
            counts = list(self.word_cnts.values())
            indexes = list(range(len(counts)))
        
        # Now we'll sort counts and while sorting them also will sort indexes.
        # We'll use those indexes to find most used N word.
        cnt = 0
        while cnt + 1 != len(counts):
            cnt = 0
            for i in range(len(counts)-1):
                if counts[i] < counts[i+1]:
                    counts[i+1],counts[i] = counts[i],counts[i+1]
                    indexes[i],indexes[i+1] = indexes[i+1],indexes[i]
                else:
                    cnt += 1
        
        for ind in indexes[:max_words]:
            self.vocab.append(temp_vocab[ind])
                    
    def textToVector(self,text):
        # First we need to split the text into its tokens and learn the length
        # If length is shorter than the max len we'll add some spaces (100D vectors which has only zero values)
        # If it's longer than the max len we'll trim from the end.
        tokens = text.split()
        len_v = len(tokens)-1 if len(tokens) < self.seq_len else self.seq_len-1
        vec = []
        for tok in tokens[:len_v]:
            try:
                vec.append(self.embed_matrix[tok])
            except Exception as E:
                pass
        
        last_pieces = self.seq_len - len(vec)
        for i in range(last_pieces):
            vec.append(np.zeros(100,))
        
        return np.asarray(vec).flatten()

In [30]:
sequencer = Sequencer(all_words = [token for seq in x_tokenized for token in seq],
              max_words = 1200,
              seq_len = 15,
              embedding_matrix = model.wv
             )

In [32]:
test_vec = sequencer.textToVector("you u got wild bitches tellin you lies")
test_vec

array([ 0.70528466,  1.21203542, -0.26196185, ...,  0.        ,
        0.        ,  0.        ])

In [33]:
test_vec.shape

(1500,)

PCA

In [34]:
# But before creating a PCA model using scikit-learn let's create
# vectors for our each vector
x_vecs = np.asarray([sequencer.textToVector(" ".join(seq)) for seq in x_tokenized])
print(x_vecs.shape)

(24783, 1500)


In [35]:
from sklearn.decomposition import PCA
pca_model = PCA(n_components=50)
pca_model.fit(x_vecs)
print("Sum of variance ratios: ",sum(pca_model.explained_variance_ratio_))



Sum of variance ratios:  0.7418678952184677


In [36]:
x_comps = pca_model.transform(x_vecs)
x_comps.shape

(24783, 50)

In [37]:


x_train,x_test,y_train,y_test = train_test_split(x_comps,y_prep,test_size=0.2,random_state=42)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)



(19826, 50)
(4957, 50)
(19826,)
(4957,)


SVM

In [38]:
start = time.time() 

svm_classifier = SVC()
svm_classifier.fit(x_train,y_train)

end = time.time()
process = round(end-start,2)
print("Support Vector Machine Classifier has fitted, this process took {} seconds".format(process))

Support Vector Machine Classifier has fitted, this process took 58.02 seconds


In [39]:
svm_classifier.score(x_test,y_test)

0.795844260641517